In [ ]:
import FeaturesText
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from transformers import AutoModel, AutoTokenizer 
import torch
import pickle 
import numpy as np
import pandas as pd 

In [ ]:
data = pd.read_csv('../data/final_training_irony.csv')

In [ ]:
txt_file = FeaturesText.preprocessing_text(data,remove_hashtags=True, remove_mentions=True, lowercase=True, arktweet_pos=True)

In [ ]:
start = time.time()
train_txt = txt_file.get_clean_df()
end = time.time()
print(end - start)

In [ ]:
train_txt = train_txt[train_txt.astype(str).pos != '[]']

In [ ]:
final_train_txt = FeaturesText.ExtractFeatures(train_txt, 'other', svd_transform=False)

### Features extraction

In [ ]:
pos, punctuation_features, emoji_features, \
onomato_features, initialism_features,\
polarity_subj_features = final_train_txt.get_all_features_train(ngram_range=(1,1), dimensionality=100)

In [ ]:
train_features = {'pos': pos,'polarity':polarity_subj_features, 'emoji': emoji_features,'punc': punctuation_features, 
                                'onom': onomato_features, 'init': initialism_features, 'label': np.asarray(train_txt.label.tolist())}

### Bert tweet embeddings

In [ ]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base",  add_special_tokens=True,
                                                max_length=50, pad_to_max_length=True,normalization=True)

In [ ]:
input_ids = [torch.tensor([tokenizer.encode(i, max_length=50, truncation=True)]) for i in train_txt.text]

In [ ]:
from tqdm import tqdm

In [ ]:
features_list = []
with torch.no_grad():
    for i in tqdm(input_ids):
        features = np.array(bertweet(i)[1]) #extract sentence embedding 1 x 768 for each document
        features_list.append(features)

In [ ]:
bert_emb = np.array(features_list).squeeze()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(bert_emb)
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
pca.fit(data_rescaled)
reduced = pca.transform(data_rescaled)
import joblib 
joblib.dump(scaler, 'scaler_embed_irony.pkl') 
joblib.dump(pca, 'pca_embedding_irony.pkl')

### Save features

In [ ]:
train_features['bert_embed'] = reduced

In [ ]:
import pickle
with open('../data/features_training_irony_twitter.p', 'wb') as fp:
    pickle.dump(train_features, fp, protocol=pickle.HIGHEST_PROTOCOL)

### Import test set

In [ ]:
sem = pd.read_csv('../data/SemEval2018-Task3/datasets/goldtest_TaskA/SemEval2018-T3_gold_test_taskA_emoji.txt', sep='\t')

In [ ]:
sem.rename({'Tweet text': 'text', 'Label' : 'label'}, axis = 1, inplace=True)
sem = pd.read_csv('../data/SemEval2018-Task3/datasets/goldtest_TaskA/SemEval2018-T3_gold_test_taskA_emoji.txt', sep='\t')

In [ ]:
test_txt= FeaturesText.preprocessing_text(sem,remove_hashtags=True, remove_mentions=True, lowercase=True, arktweet_pos=True)

In [ ]:
start = time.time()
test_txt_sem = test_txt.get_clean_df()
end = time.time()
print(end - start)

In [ ]:
pos_sem, punctuation_features_sem, emoji_features_sem, \
onomato_features_sem, initialism_features_sem,\
polarity_subj_features_sem = FeaturesText.ExtractFeatures.get_all_features_test(final_train_txt, test_set=test_txt_sem)

In [ ]:
test_features_sem = {'pos': pos_sem,'polarity':polarity_subj_features_sem, 'emoji': emoji_features_sem,'punc': punctuation_features_sem, 
                                'onom': onomato_features_sem, 'init': initialism_features_sem, 'label': np.asarray(sem.label.tolist())}

In [ ]:
input_ids_sem = [torch.tensor([tokenizer.encode(i, max_length=50, truncation=True)]) for i in sem.text]

In [ ]:
features_list_sem = []
with torch.no_grad():
    for i in tqdm(input_ids_sem):
        features = np.array(bertweet(i)[1]) #extract sentence embedding 1 x 768 for each document
        features_list_sem.append(features)

In [ ]:
bert_emb_sem = np.array(features_list_sem).squeeze()

In [ ]:
# later reload the pickle file
import pickle as pk
import joblib
pca_reload = joblib.load("pca_embedding_irony.pkl")
standar_reload = joblib.load("scaler_embed_irony.pkl")

 
#result_new = pca_reload.transform(X)

In [ ]:
std_sem = standar_reload.transform(bert_emb_sem)

In [ ]:
reduced_sem = pca_reload.transform(std_sem)

In [ ]:
test_features_sem['bert_embed'] = reduced_sem

In [ ]:
with open('../data/semeval_test3a_irony.p', 'wb') as fp:
    pickle.dump(test_features_sem, fp, protocol=pickle.HIGHEST_PROTOCOL)